# INSTALL: packages

In [1]:
pip install nbformat

^C
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# EXECUTE: WalletClustering_AddressIdentification notebook

In [2]:
%run ./WalletClustering_AddressIdentification.ipynb # includes list creation of addresses
# methods & variables of notebook can be referenced

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
# only to remember lists
terrorAddressList = pickle.load(open('output/terrorAddressList.pickle', 'rb'))# list of extracted flagged terror addresses
blacklistAddresses = pickle.load(open('output/blacklistAddresses.pickle', 'rb')) # list of exchange wallet addresses
addressesToCluster = pickle.load(open('output/addressesToCluster.pickle', 'rb')) # list of flagged terror addresses without blacklist addresses

# DEFINE: weak signal heuristic methods to cluster wallets

### build hint network with MI heuristic

In [4]:
mihTemplate = '''
MATCH (:Address{address:"%s"})-[:SENDS]->(t:Transaction),
(walletMember:Address)-[:SENDS]->(t:Transaction)
RETURN DISTINCT walletMember.address
'''

In [5]:
def multiInputClustering(address, walletAddresses):
    walletAddresses.append(address)
    response = conn.query(mihTemplate % address, db='neo4j')
    
    newAddresses = [r[0] for r in response]
    newAddresses = [a for a in newAddresses if a not in walletAddresses]
    walletAddresses += newAddresses
    
    for a in newAddresses:
        walletAddresses += multiInputClustering(a, walletAddresses)
        
    return list(set(walletAddresses))

In [6]:
def iterMultiInputClustering(addressList, alreadyClusteredAddresses):
    walletOverview=[]
    clusteredAddresses = alreadyClusteredAddresses
     
    for address in addressList:
        if address not in clusteredAddresses:
            wallet = multiInputClustering(address, [])
            wallet.sort()
            walletHead = wallet[0]
            walletOverview.append([walletHead, wallet])
            clusteredAddresses.extend(wallet)

    walletOverview.sort()
    clusteredAddresses.sort()

    return walletOverview, clusteredAddresses


### wallet graph

In [ ]:
# def mihWalletGraphing(mihOutput):
#     for wallet in mihOutput:
#         insert here: build node for head-address (user-id)

In [8]:
# get all addresses associated with MI-clusters via transaction output

WshNetworkTemplate = '''
MATCH (:Address{address:"%s"})-[:SENDS]->(t:Transaction)-[:RECEIVES]->(maybeMember:Address)
RETURN maybeMember.address
'''

In [10]:
def getPotentials(mihOutput,mihAddresses):
    extendedAddresses=[]
    for wallet in mihOutput:
        for address in wallet[1]:
            # query additional nodes
            response = conn.query(WshNetworkTemplate % address, db='neo4j')
            for entry in response:
                extendedAddresses.append(dict(entry)['maybeMember.address'])  # not yet optimal
    
    extendedAddresses = set(extendedAddresses)
    
    extendedAddresses = addressCleanUp(extendedAddresses,mihAddresses)
    
    extendedAddresses = addressCleanUp(extendedAddresses,blacklistAddresses) # currently very slow
    
    # so far, only one extension layer. could also be implemented iteratively until extended addresses is 0 -> very expensive
    with open('blacklistAddresses.pickle', 'wb') as export:
        pickle.dump(blacklistAddresses, export)
        
    return extendedAddresses

In [ ]:
# To be implemented
# def potentialsGraphing(mihGraph, potentials):
#     for wallet in potentials:
#         insert here: build node for head-address of pontential (user-id)

In [ ]:
# To be implemented: for each transaction, output in-addresses and out-addresses as one record

WshTemplate = '''
MATCH (:Address{address:"%s"})-[:SENDS]->(t:Transaction)-[:RECEIVES]->(maybeMember:Address)
RETURN maybeMember.address
'''

In [ ]:
# To be implemented
# transform transactions into user-->users transactions
# def userToUserTransactions(mihOutput)

In [ ]:
# To be implemented
# def walletAssociation(graphOfWallets, mihOutput)
#   get tranformed transactions via userToUserTransactions
#   add edges among users if <10 output-users and all output-users different from input-user

### apply community detection

In [ ]:
# To be implemented
# via Louvain algorithm on level 1 aggregation

### flag clusters

In [11]:
def flagAdditionalTerrorAdresses(input):
    query = """
    MATCH (a:Address {address: "%s"})
    set a.isTerrorMultiInput = True,
    a.terrorWallet = "%s"
    Return a.address
    """
    indexWallets = 0
    while indexWallets < len(input):
        for address in input[indexWallets]:
            walletID = input[indexWallets+1]
            conn.query(query % (address, walletID), db='neo4j')
        indexWallets += 2

# EXECUTE: weak signal heuristic methods to cluster wallets

In [12]:
# MIH-cluster input addresses

testWallets, knownAddresses = iterMultiInputClustering(['15K9Zj1AU2hjT3ebZMtWqDsMv3fFxTNwpf','17UUXDzPGkMwWrabhtk7YCha88tSoua2Vr', '1LNhqAoXZJ25pr4R6Zg4BRHqWe8umooFET'],[])
testWallets

[['14GqbBbquzTQJXirgpdZhgqGHQVk1wmVSz',
  ['14GqbBbquzTQJXirgpdZhgqGHQVk1wmVSz',
   '15QZXBQvC3s415UGyNVeZc5EwvX9JstwqS',
   '17UUXDzPGkMwWrabhtk7YCha88tSoua2Vr',
   '18fqEqikTekFgKYKY1GMgZSUtDKw3Hu973',
   '18yuLP1djW1q1A235BBAqXcjC5KkhqkM2M',
   '1C53nhWp54gyCRtspd1iZHm7jFUCsK3tLW',
   '1C6hetVWVXZnS6P2BYBNu5Y1ZJ57JyXGac',
   '1FYXRzE7CeZFANe2Vre5F5NA6Yw2WDCjdu',
   '1FaMA59S8qXppytCdfhmqMGrFww3gVP2Ld',
   '1Fv9pEor3XVFsWU2PWKanD6S2QrBEGe943',
   '1JbEKTZciGaBjcpz7RzxmH8AVQ4tYZwt9w',
   '1K3qN1fQzQzcM3adRV3MJbTjxfAVg81xgR',
   '1LNhqAoXZJ25pr4R6Zg4BRHqWe8umooFET',
   '1MBpXWxP8bT7aRixpRYCMrAPedSxuqB8mp',
   '1Mrn5JmzCDvK2UecBR6Nwhf9xVCjBBoCP2']],
 ['15K9Zj1AU2hjT3ebZMtWqDsMv3fFxTNwpf',
  ['15K9Zj1AU2hjT3ebZMtWqDsMv3fFxTNwpf',
   '1BQAPyku1ZibWGAgd8QePpW1vAKHowqLez']]]

In [13]:
# graph nodes of input addresses

# walletGraph = mihWalletGraphing (testWallets)

In [14]:
potentialAddresses = getPotentials(testWallets, knownAddresses)
potentialAddresses

['15BkDAkpkKq5PfogEmyneNXiZzUXBWDsQN',
 '16YHAdaD589hV4nLDVUTEFcgaDpvTncQg2',
 '1Fic1r8Ncj3YKRUjv5SgBnetMv5vPmWfkR',
 '1GMB5HWPQSk2C9mzH7yPAZTz61Au8KW6Zv',
 '12RkDDkekjF5QtBy4vZxVvyQiMam9eS2wK',
 '1H9zEVipYuFkB1ZXfNqFF1JtJKpn6vfhdk',
 '162sVcSd23yjGVALXuTYS5rt6YtYU5555q']

In [15]:
potentialWallets, knownAddresses = iterMultiInputClustering(potentialAddresses, knownAddresses)
potentialWallets

[['112oNT2YhN5WYvSSAqGL8YYGkz46rSJCJ2',
  ['112oNT2YhN5WYvSSAqGL8YYGkz46rSJCJ2',
   '112ysxdPYPsPjHTG47KMhGd9ULYdG5fCj1',
   '115z3BaTWYRJbHZYMZVLxetSgqeQ28GrXj',
   '121iEyvAdjQU2PDyxYNFdzZnkKSLDGNb67',
   '123fBH3GCZxp1Cz7xrUdguSfxvVAugZQJo',
   '124b1CEF536Chudc1LqzdvBSoXri2nkVn6',
   '125zvDU3mCsQNR5o9LrpxRMQnEaRP4VKh8',
   '126PW119CbUDZsxXx828aqLuQQyPSdwVKS',
   '127mnTHED2ZqXjtYqkwaLJPQSyUqccShrT',
   '127uNvPhKLHy4KAkdR79oYdZLgSvBcu4Pu',
   '129M4ruGioxPfiYU9dHkkaEG7kXQhPvZqy',
   '12EGpqwabMuzjTDMNhhQkmKJgiKx2gX8ti',
   '12GvVeCQjgjPBe8NZqfX5sqwAqvDJNGrWe',
   '12JSwfhcCvX8oUv4WqJvA16K448t9hkf8s',
   '12KSQThfaMdXY8yDuft2739NzZPkR52LDD',
   '12MCFZ1YnuzS6ysgbKgDJBvTB1RJvw6h9T',
   '12MyVuqTZH8isoDpGvH9KSauRzUQF65pXk',
   '12NghSnCietqj67omrXnBrsZLR8wkKhRDV',
   '12QZ6Cq7jg4bWEkNWK6RnvzLDTRDL3YnAf',
   '12SzHFGY3eeZmq8MBB77AJvzPm8SAzHfio',
   '12TJpdkExoqfg4kidPc5UKET8S2sdEPzbF',
   '12TWAarP2mQwpn72xmHbVemZACP6wBAoCy',
   '12TWaaqFGD2stKtQJTPyzP6ty7zXq4PuEQ',
   '12WBNANqVfAmq

In [16]:
# graph nodes of extended address network
 
# extendedWalletGraph = potentialsGraphing(walletGraph, potentialWallets)

In [ ]:
# add edges among users 

# walletGraphWithEdges = walletAssociation(extendedWalletGraph, testWallets)

In [17]:
# perform community detection on extended network 

# finalCluster = communityDetection(walletGraphWithEdges)

In [ ]:
# eventually flag addresses in Neo4j